# 抓買超賣超
沒有用滾輪按按鈕選日期

In [1]:
import time, os, pickle
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np

import datetime
from datetime import timedelta

#! pip install chinesecalendar
import chinese_calendar
from chinese_calendar import is_workday

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time
chromedriver_autoinstaller.install()
chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(options=chrome_options)

Wall time: 4.63 s


## 1. 抓買超賣超表格 fun

In [19]:
def table(ticker):
    '''
    回傳: 
    1. 買超表格
    2. 賣超表格
    3. 重要東東的值
    4. 重要東東的名字
    '''
    html = browser.page_source
    soup = bs(html, "lxml")
    
    try:
        # 值
        values = soup.find_all('td', "t3n1")
        value_list = [value.getText() for value in values]

        # 券商
        names = soup.find_all('td', "t4t1") 
        name_list = [name.getText() for name in names]

        # 欄位名稱
        titles = soup.find_all('td', "t2")  
        title_list = [title.getText() for title in titles][-10:]

        # 日期
        date = soup.find('div', "t11").getText()[-10:] 
        date = '-'.join(date.split('/'))

        # 將 dataframe 分為買超賣超 
        alldf = pd.DataFrame(np.array(value_list[:-4]).reshape(-1, 8)) 
        df_buy = alldf.iloc[:, :4]
        df_sell = alldf.iloc[:, 4:]

        # + 券商名
        buy = [i for i in range(len(name_list[:-4])) if i%2==0 ] 
        sell = [i for i in range(len(name_list[:-4])) if i%2!=0 ]
        df_buy.insert(0, '買超券商', [name_list[i] for i in buy])  
        df_sell.insert(0, '賣超券商', [name_list[i] for i in sell])

        # + 欄位名
        df_buy.columns = title_list[:5] 
        df_sell.columns = title_list[-5:]

        # + 資料日期
        df_buy.insert(0, "資料日期", date)  
        df_sell.insert(0, "資料日期", date)

        # + 證券代號
        df_buy.insert(1, "證券代號", ticker)  
        df_sell.insert(1, "證券代號", ticker)

        return df_buy, df_sell, value_list, name_list, date
    
    except Exception as e:
        print('【系統訊息】{}'.format(e))
        return None
        

##  2. 彙整 fun
匯出2個檔案
1. 每個日期各一個買超賣超 excel 檔 (2 sheet)      `ticker-日期.xlsx`
2. 重要東東 所有日期合成一個 csv 檔                `ticker-OverBuySale.csv`

In [20]:
def OverBuySale(ticker, tillnow=0, year=datetime.datetime.now().date().year, 
                month=datetime.datetime.now().date().month, 
                day=datetime.datetime.now().date().day, start_end=None):
    '''
    ticker: (int) 證券代號
    tillnow: (int) 輸入要抓的前 or 後幾個工作天(根據基準日期) ex: 前3天(包含今天共4天):-3
    year: (int) 基準年份
    month: (int) 基準月份
    day: (int) 基準日期
    start_end: (list) 格式:[起始年, 起始月, 起始日, 終止年, 終止月, 終止日]
    
    **default: 抓當天**
    
    return 
    df_summary: 重要的表格
    problem: 查無資料的日期
    '''
    
    # 建立存檔案的資料夾
    path_go1 = f'{os.getcwd()}\\買超賣超-主力進出明細表\\{str(ticker)}'   
    if not os.path.exists(path_go1) : 
        os.makedirs(path_go1)
    path_go2 = f'{os.getcwd()}\\合計平均買賣超'   
    if not os.path.exists(path_go2) : 
        os.makedirs(path_go2)
            
    # 找出工作天
    if start_end==None:
        find_work = [chinese_calendar.find_workday(delta_days=tillnow, date=datetime.date(year, month, day)), datetime.date(year, month, day)]
        work = chinese_calendar.get_workdays(min(find_work), max(find_work))
    else:
        work = chinese_calendar.get_workdays(datetime.date(start_end[0], start_end[1], start_end[2]), 
                                             datetime.date(start_end[3], start_end[4], start_end[5]))
    
    # 要抓的年份-月份-日 
    years = [work[i].year for i in range(len(work))]
    months = [work[i].month for i in range(len(work))]
    days = [work[i].day for i in range(len(work))]
    
    all_choose = list(zip(years, months, days))
    #print(ticker, '要抓的工作天有', [str(m)+'/'+str(d) for _,m,d in all_choose])
    
    # 單支股票所有日期的重要東東
    df_summary = pd.DataFrame(columns = ['資料日期', '證券代號', '合計買超張數', '合計賣超張數', '平均買超成本', '平均賣超成本'])
    
    problem = []  ## 存下查無資料的日期
    for choose in all_choose:
        
        y, m, d = choose
        url = f""https://stockchannelnew.sinotrade.com.tw/z/zc/zco/zco.djhtm?a={ticker}&e={y}-{m}-{d}&f={y}-{m}-{d}
        browser.get(url)
        time.sleep(2)
        
        # 抓表格 
        tabletable = table(ticker)
        if isinstance(tabletable, type(None)):
            problem.append([str(y)+'/'+str(m)+'/'+str(d)])
            print(f'{y}/{m}/{d} 的資料 error')
            continue
        else:
            df_buy, df_sell, value_list, name_list, date = tabletable

            if len(df_buy)!=0 and len(df_sell)!=0:

                # **匯出**單日 excel買超賣超表格 (各一個 sheet)     
                filename = str(ticker) + "-" + ''.join(str(date).split('-')) + str(".xlsx")

                writer = pd.ExcelWriter(f'{path_go1}\\{filename}')   
                i=1
                for file in [df_buy, df_sell]:
                    file.to_excel(writer, sheet_name = '買超' if i==1 else '賣超', index=False)
                    i+=1
                writer.save()
                writer.close()
                del i
                print(f'成功匯出 {filename}')
            else:
                problem.append([str(y)+'/'+str(m)+'/'+str(d)])
                print(f'查無 {y}/{m}/{d} 的資料')
                continue

            # 單支股票所有日期的重要東東
            new = dict(zip(name_list[-4:], value_list[-4:]))
            new['資料日期'] = date
            new['證券代號'] = ticker
            df_summary = df_summary.append(new, ignore_index=True)
        
    # **匯出**單支股票所有日期的重要東東
    filename2 =str(ticker) + "-" + 'OverBuySale' + str(".csv")
    df_summary.to_csv(f'{path_go2}\\{filename2}', encoding='cp950', index = False)
    print(f'成功匯出 {filename2}')

    
    return df_summary, problem

In [21]:
OverBuySale(3529, start_end=[2022, 4, 24, 2022, 4, 26])

查無 2022/4/24 的資料
成功匯出 3529-20220425.xlsx
成功匯出 3529-20220426.xlsx
成功匯出 3529-OverBuySale.csv


(         資料日期  證券代號 合計買超張數 合計賣超張數    平均買超成本    平均賣超成本
 0  2022-04-25  3529    707    773  1,133.84  1,132.22
 1  2022-04-26  3529    931    956  1,091.64  1,092.96,
 [['2022/4/24']])

### 範例

In [77]:
# ex: 直接指定 抓 6/28~ 7/5 (今天是 7/7)
OverBuySale(8027, start_end=[2022, 6, 28, 2022, 7, 5])
print('')

# ex: 抓 6/28 前 2 個工作天 (包含 628 所以是 3 天....)
OverBuySale(8027, tillnow = -2, month = 6, day = 28)
print('')

# ex: 抓 6/28 後 2 個工作天 (包含 628 所以是 3 天....)
OverBuySale(8027, tillnow = 2, month = 6, day = 28)
print('')

# ex: 抓當天
OverBuySale(8027)
print('')

# ex: 抓今天的前 2 個工作天 (包含今天所以是 3 天....)
OverBuySale(8027, tillnow = -2)

成功匯出 8027-20220628.xlsx
成功匯出 8027-20220629.xlsx
成功匯出 8027-20220630.xlsx
成功匯出 8027-20220701.xlsx
成功匯出 8027-20220704.xlsx
成功匯出 8027-20220705.xlsx
成功匯出 8027-OverBuySale.csv

成功匯出 8027-20220624.xlsx
成功匯出 8027-20220627.xlsx
成功匯出 8027-20220628.xlsx
成功匯出 8027-OverBuySale.csv

成功匯出 8027-20220628.xlsx
成功匯出 8027-20220629.xlsx
成功匯出 8027-20220630.xlsx
成功匯出 8027-OverBuySale.csv

查無 2022/7/13 的資料
成功匯出 8027-OverBuySale.csv

成功匯出 8027-20220711.xlsx
成功匯出 8027-20220712.xlsx
查無 2022/7/13 的資料
成功匯出 8027-OverBuySale.csv


(         資料日期  證券代號 合計買超張數 合計賣超張數 平均買超成本 平均賣超成本
 0  2022-07-11  8027    107    121  62.45  62.42
 1  2022-07-12  8027    231    325  59.67  59.94,
 [['2022/7/13']])

## 3. 抓好幾支股票

### 要爬的股票

In [7]:
path_from = os.getcwd() 
tickers = pd.read_csv(path_from + r'\台股全部4.csv', header=None)[0].tolist()

### 設定開始爬的股票號碼

In [10]:
start = 4142  # 自己輸入

start_index = int([i for (i, j) in enumerate(tickers) if j == start][0])
print(f'從第 {start_index+1} 個開始')

從第 5 個開始


In [11]:
time_start = time.time() #開始計時

# 打開之前儲存的錯誤 dict 繼續沿用 (optional)，留下查無資料的股票號碼 & 日期
if not os.path.exists(r'2_problem_dict.pkl') :
    problem_dict = {}
else:
    with open(r'2_problem_dict.pkl', mode='rb') as inpf :  # 打開之前儲存的錯誤 dict 繼續沿用
        try:
            problem_dict = pickle.load(inpf)
        except EOFError:
            problem_dict = {}

            
#tickers = [8027, 4961, 4967]     ## 設定股票代碼        
for ticker in tickers[start_index:]:
    df_summary, problem = OverBuySale(ticker=ticker, tillnow = -30)  # 自己輸入
    
    # 如果有查無資料的話
    if len(problem)!=0:
        # 存下查無資料的股票號碼 & 日期 (dict) 
        problem_dict[ticker] = problem

        # 匯出查無資料的股票號碼 & 日期
        with open(r'2_problem_dict.pkl', mode='wb') as outf :
            pickle.dump(problem_dict, outf)
        print(f'已將查無資料的股票號碼 & 日期匯出為 2_problem_dict.pkl') 
         
    print(f'爬完 {ticker} ~~~')
    
    time_end = time.time() 
    timespend= (time_end- time_start)/60
    print('spend '+ str(round(timespend, 2))+ ' min')
    print('')
 

成功匯出 4142-20220525.xlsx
成功匯出 4142-20220526.xlsx
成功匯出 4142-20220527.xlsx
成功匯出 4142-20220530.xlsx
成功匯出 4142-20220531.xlsx
成功匯出 4142-20220601.xlsx
成功匯出 4142-20220602.xlsx
成功匯出 4142-20220606.xlsx
成功匯出 4142-20220607.xlsx
成功匯出 4142-20220608.xlsx
成功匯出 4142-20220609.xlsx
成功匯出 4142-20220610.xlsx
成功匯出 4142-20220613.xlsx
成功匯出 4142-20220614.xlsx
成功匯出 4142-20220615.xlsx
成功匯出 4142-20220616.xlsx
成功匯出 4142-20220617.xlsx
成功匯出 4142-20220620.xlsx
成功匯出 4142-20220621.xlsx
成功匯出 4142-20220622.xlsx
成功匯出 4142-20220623.xlsx
成功匯出 4142-20220624.xlsx
成功匯出 4142-20220627.xlsx
成功匯出 4142-20220628.xlsx
成功匯出 4142-20220629.xlsx
成功匯出 4142-20220630.xlsx
成功匯出 4142-20220701.xlsx
成功匯出 4142-20220704.xlsx
成功匯出 4142-20220705.xlsx
成功匯出 4142-20220706.xlsx
查無 2022/7/7 的資料
成功匯出 4142-OverBuySale.csv
已將查無資料的股票號碼 & 日期匯出為 2_problem_dict.pkl
爬完 4142 ~~~
spend 1.2 min

成功匯出 4147-20220525.xlsx
成功匯出 4147-20220526.xlsx
成功匯出 4147-20220527.xlsx
成功匯出 4147-20220530.xlsx
成功匯出 4147-20220531.xlsx
成功匯出 4147-20220601.xlsx


KeyboardInterrupt: 

### 偷看

In [80]:
# 單支股票所有日期的重要東東 大致長醬
OverBuySale(8027, tillnow = -2)[0]

成功匯出 8027-20220711.xlsx
成功匯出 8027-20220712.xlsx
查無 2022/7/13 的資料
成功匯出 8027-OverBuySale.csv


,資料日期,證券代號,合計買超張數,合計賣超張數,平均買超成本,平均賣超成本
0,2022-07-11,8027,107,121,62.45,62.42
1,2022-07-12,8027,231,325,59.67,59.94


In [13]:
# 目前查無資料的股票號碼 & 日期
print(f'目前查無資料: {problem_dict}') 

目前查無資料: {4585: [['2022/7/7']], 8027: [['2022/7/7']], 4961: [['2022/7/7']], 4967: [['2022/7/7']], 5006: [['2022/5/25'], ['2022/5/26'], ['2022/5/27'], ['2022/5/30'], ['2022/5/31'], ['2022/6/1'], ['2022/6/2'], ['2022/6/6'], ['2022/6/7'], ['2022/6/8'], ['2022/6/9'], ['2022/6/10'], ['2022/6/13'], ['2022/6/14'], ['2022/6/15'], ['2022/6/16'], ['2022/6/17'], ['2022/6/20'], ['2022/6/21'], ['2022/6/22'], ['2022/6/23'], ['2022/6/24'], ['2022/6/27'], ['2022/6/28'], ['2022/6/29'], ['2022/6/30'], ['2022/7/1'], ['2022/7/4'], ['2022/7/5'], ['2022/7/6'], ['2022/7/7']], 4142: [['2022/7/7']]}
